# TEPEDELEN Léo DELLARICA Steven BARBIN Kevin

# Projet EISI1 Capgemini MSPRTPRE813_1

Ce notebook Python est le rapport détaillé et l'ETL de notre projet. La suite du rapport se situe après l'ETL, nous y détaillons le fonctionnement du reste du système.

Il a été réalisé à l'aide de Docker afin d'en faciliter le déploiement pour tester et collaborer.

Vous trouverez tout ce qu'il vous faut pour essayer le programme sur votre machine dans le fichier README de ce repository github :

# Extraire les données dans un DataFrame pySpark

In [1]:
import pandas as pd

In [2]:
from pyspark.sql import functions as F

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Créer une session Spark
spark = SparkSession.builder.appName("example") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "1") \
    .config("spark.dynamicAllocation.maxExecutors", "50") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.memory.fraction", "0.8") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [4]:
delinquance_brut = pd.read_parquet("https://www.dropbox.com/scl/fi/gnuphrytsuw0d1rx2ix0i/donnee-comm-data.gouv-parquet-2023-geographie2024-produit-le2024-07-05.parquet?rlkey=hm2t98tm3vyzks7q0ehaoh94e&st=4gttfgqe&dl=1", engine='pyarrow')

In [5]:
delinquance_brut.drop(columns=['millPOP','millLOG','classe', 'unité.de.compte',	'valeur.publiée', 'complementinfoval', 'complementinfotaux', 'POP', 'LOG'], inplace=True)

In [6]:
delinquance_brut.rename(columns={'CODGEO_2024' : 'codecommune', 'faits':'crimes_delits'}, inplace=True)

In [7]:
delinquance_brut.dropna(subset=['annee'], inplace=True)

In [8]:
delinquance_brut['codecommune'] = delinquance_brut['codecommune'].str.replace('[^0-9]', '', regex=True).astype('int64')

In [9]:
delinquance_brut['annee'] = delinquance_brut['annee'].astype(str)
delinquance_brut['annee'] = delinquance_brut['annee'].str.replace('.0','')
delinquance_brut['annee'] = '20' + delinquance_brut['annee']
delinquance_brut['annee'] = delinquance_brut['annee'].str.replace('[^0-9]', '', regex=True).astype('int64')

In [10]:
delinquance_brut = delinquance_brut.groupby(['codecommune','annee'])['crimes_delits'].sum().reset_index()

In [11]:
delinquance_brut['crimes_delits'] = delinquance_brut['crimes_delits'].astype('int64')

In [12]:
delinquance_brut = spark.createDataFrame(delinquance_brut)

In [13]:
# candidats_results.parquet
df_election = pd.read_parquet('https://www.dropbox.com/scl/fi/04jpbz2gqcax0y8ubetz8/candidats_results.parquet?rlkey=cvs3816ot12wxg41ss02q4aag&st=0r4ilpa8&dl=1')

In [14]:
def fillNuance(df, col, value, nuance):
    df.loc[df[col] == value, ["Nuance"]] = nuance

In [15]:
missing_nuances_libelle = {
    "ALLIANCE JAUNE":"EGAUCHE",
    "ALLONS ENFANTS":"GAUCHE",
    "DEBOUT LA FRANCE":"EDROITE",
    "DÉCROISSANCE 2019":"EGAUCHE",
    "DÉMOCRATIE REPRÉSENTATIVE":"EGAUCHE",
    "ENSEMBLE PATRIOTES":"EDROITE",
    "ENSEMBLE POUR LE FREXIT":"EDROITE",
    "ENVIE D'EUROPE":"GAUCHE",
    "ESPERANTO":"EGAUCHE",
    "EUROPE AU SERVICE PEUPLES":"GAUCHE",
    "EUROPE ÉCOLOGIE":"GAUCHE",
    "INITIATIVE CITOYENNE":"EGAUCHE",
    "LA FRANCE INSOUMISE":"GAUCHE",
    "LA LIGNE CLAIRE":"EDROITE",
    "LES EUROPÉENS":"CENTRE",
    "LES OUBLIES DE L'EUROPE":"CENTRE",
    "LISTE CITOYENNE":"GAUCHE",
    "LISTE DE LA RECONQUÊTE":"EDROITE",
    "LUTTE OUVRIÈRE":"EGAUCHE",
    "NEUTRE ET ACTIF":"EDROITE",
    "PACE":"GAUCHE",
    "PARTI ANIMALISTE":"CENTRE",
    "PARTI FED. EUROPÉEN":"EDROITE",
    "PARTI PIRATE":"GAUCHE",
    "POUR L'EUROPE DES GENS":"EGAUCHE",
    "PRENEZ LE POUVOIR":"EDROITE",
    "RENAISSANCE":"CENTRE",
    "RÉVOLUTIONNAIRE":"EGAUCHE",
    "UDLEF":"DROITE",
    "UNE FRANCE ROYALE":"EDROITE",
    "UNION DROITE-CENTRE":"CDROIT",
    "URGENCE ÉCOLOGIE":"EGAUCHE",
    "À VOIX ÉGALES":"GAUCHE",
    "ÉVOLUTION CITOYENNE":"GAUCHE"
}

In [16]:
missing_nuances_nom = {
    "ARTHAUD":"EGAUCHE",
    "ASSELINEAU":"EDROITE",
    "CHEMINADE":"EDROITE",
    "DUPONT-AIGNAN":"EDROITE",
    "FILLON":"DROITE",
    "HAMON":"GAUCHE",
    "HIDALGO":"GAUCHE",
    "JADOT":"GAUCHE",
    "LASSALLE":"DROITE",
    "LE PEN":"EDROITE",
    "MACRON":"CENTRE",
    "MÉLENCHON":"GAUCHE",
    "POUTOU":"EGAUCHE",
    "PÉCRESSE":"DROITE",
    "ROUSSEL":"EGAUCHE",
    "ZEMMOUR":"EDROITE"
}

In [17]:
for libelle, nuance in missing_nuances_libelle.items():
    fillNuance(df_election, "Libellé Abrégé Liste", libelle, nuance)

for nom, nuance in missing_nuances_nom.items():
    fillNuance(df_election, "Nom", nom, nuance)

In [18]:
del missing_nuances_nom
del missing_nuances_libelle

In [19]:
import urllib.request
import json
with urllib.request.urlopen('https://www.dropbox.com/scl/fi/trxktezbq12phyqptj545/nuances.json?rlkey=5u4368x6evhct81slh20ffzii&st=mw31zc2i&dl=1') as outfile:
     dic_nuances = json.load(outfile)

In [20]:
def replaceValue(nom_nuance, dic_nuances):
    for nuance, list_nom_nuance in dic_nuances.items():
        if nom_nuance in list_nom_nuance:
            return nuance
    return nom_nuance

df_election['Nuance'] = df_election['Nuance'].apply(lambda nom_nuance: replaceValue(nom_nuance, dic_nuances))

In [21]:
del dic_nuances

In [22]:
df_election = df_election[~df_election["id_election"].str.contains(r'muni|dpmt|regi|cant', na=False)]

In [23]:
df_election = df_election[["id_election","Code du département","Code de la commune", "Voix","Code du b.vote", "% Voix/Ins", "Nuance"]]

In [24]:
df_election[["annee", "type_election", "tour"]] = df_election["id_election"].str.split("_", expand=True)

In [25]:
# df_election["Code de la commune"] = df_election.apply(lambda row: row["Code du département"] + row["Code de la commune"], axis=1)
df_election['Code de la commune'] = df_election['Code du département'] + df_election['Code de la commune']

In [26]:
df_election["Code de la commune"] = pd.to_numeric(df_election["Code de la commune"], errors='coerce')

In [27]:
df_election.dropna(subset=["Code du département", "Code de la commune"], inplace=True)
df_election["Code de la commune"] = df_election["Code de la commune"].astype(int)

In [28]:
df_election.rename(columns={
    "Code du département":"dep",
    "Code de la commune":"codecommune",
    "Code du b.vote":"bureau",
    "% Voix/Ins":"pourcentage_voix_inscrits",
    "Nuance":"nuance",
    "Voix":"nb_voix"
}, inplace=True)

In [29]:
df_election = spark.createDataFrame(df_election).cache()

In [30]:
df_election = df_election.groupBy("id_election", "codecommune", "bureau", "annee", "type_election", "tour").pivot("nuance").agg(F.sum("pourcentage_voix_inscrits")) \
                        .select(
                            "id_election",
                            "codecommune",
                            "bureau",
                            "annee",
                            "type_election",
                            "tour",
                            col("AUTRE").alias("autre_pourcentage_voix_inscrits"),
                            col("EGAUCHE").alias("egauche_pourcentage_voix_inscrits"),
                            col("GAUCHE").alias("gauche_pourcentage_voix_inscrits"),
                            col("CGAUCHE").alias("cgauche_pourcentage_voix_inscrits"),
                            col("CENTRE").alias("centre_pourcentage_voix_inscrits"),
                            col("CDROIT").alias("cdroit_pourcentage_voix_inscrits"),
                            col("DROITE").alias("droite_pourcentage_voix_inscrits"),
                            col("EDROITE").alias("edroite_pourcentage_voix_inscrits")
                       )\
                        .join(
                            df_election.groupBy("id_election", "codecommune", "bureau", "annee", "type_election", "tour").pivot("nuance").agg(F.sum("nb_voix"))\
                            .select(
                                "id_election",
                                "codecommune",
                                "bureau",
                                "annee",
                                "type_election",
                                "tour",
                                col("AUTRE").alias("autre_nb_voix"),
                                col("EGAUCHE").alias("egauche_nb_voix"),
                                col("GAUCHE").alias("gauche_nb_voix"),
                                col("CGAUCHE").alias("cgauche_nb_voix"),
                                col("CENTRE").alias("centre_nb_voix"),
                                col("CDROIT").alias("cdroit_nb_voix"),
                                col("DROITE").alias("droite_nb_voix"),
                                col("EDROITE").alias("edroite_nb_voix")
                            ), ["id_election", "codecommune", "bureau", "annee", "type_election", "tour"]
                        )

In [31]:
colonnes_a_remplir = [
    "autre_pourcentage_voix_inscrits",
    "egauche_pourcentage_voix_inscrits",
    "gauche_pourcentage_voix_inscrits",
    "cgauche_pourcentage_voix_inscrits",
    "centre_pourcentage_voix_inscrits",
    "cdroit_pourcentage_voix_inscrits",
    "droite_pourcentage_voix_inscrits",
    "edroite_pourcentage_voix_inscrits",
    "autre_nb_voix",
    "egauche_nb_voix",
    "gauche_nb_voix",
    "cgauche_nb_voix",
    "centre_nb_voix",
    "cdroit_nb_voix",
    "droite_nb_voix",
    "edroite_nb_voix"
]
df_election = df_election.fillna(0, subset=colonnes_a_remplir)

del colonnes_a_remplir

df_election = df_election.withColumn("nb_voix", F.expr(
    "autre_nb_voix \
    + egauche_nb_voix \
    + gauche_nb_voix \
    + cgauche_nb_voix \
    + centre_nb_voix \
    + cdroit_nb_voix \
    + droite_nb_voix \
    + edroite_nb_voix"
))

df_election = df_election.withColumn("pourcentage_voix_inscrits", F.expr(
    "autre_pourcentage_voix_inscrits \
    + egauche_pourcentage_voix_inscrits \
    + gauche_pourcentage_voix_inscrits \
    + cgauche_pourcentage_voix_inscrits \
    + centre_pourcentage_voix_inscrits \
    + cdroit_pourcentage_voix_inscrits \
    + droite_pourcentage_voix_inscrits \
    + edroite_pourcentage_voix_inscrits"
))

In [32]:
df_election = df_election.withColumn("nb_inscrits", F.round(F.expr("(nb_voix * 100) / pourcentage_voix_inscrits"), 0))

In [33]:
df_election = df_election.drop("bureau").groupBy("id_election", "codecommune", "annee", "type_election", "tour") \
    .agg(
        F.sum("nb_inscrits").alias("nb_inscrits"),
        F.sum("nb_voix").alias("nb_voix"),
        F.sum("autre_nb_voix").alias("autre_nb_voix"),
        F.sum("egauche_nb_voix").alias("egauche_nb_voix"),
        F.sum("gauche_nb_voix").alias("gauche_nb_voix"),
        F.sum("cgauche_nb_voix").alias("cgauche_nb_voix"),
        F.sum("centre_nb_voix").alias("centre_nb_voix"),
        F.sum("cdroit_nb_voix").alias("cdroit_nb_voix"),
        F.sum("droite_nb_voix").alias("droite_nb_voix"),
        F.sum("edroite_nb_voix").alias("edroite_nb_voix")
    )\
    .withColumn("autre_pourcentage_voix_inscrits", (F.col("autre_nb_voix") / F.col("nb_inscrits")) * 100) \
    .withColumn("egauche_pourcentage_voix_inscrits", (F.col("egauche_nb_voix") / F.col("nb_inscrits")) * 100) \
    .withColumn("gauche_pourcentage_voix_inscrits", (F.col("gauche_nb_voix") / F.col("nb_inscrits")) * 100) \
    .withColumn("cgauche_pourcentage_voix_inscrits", (F.col("cgauche_nb_voix") / F.col("nb_inscrits")) * 100) \
    .withColumn("centre_pourcentage_voix_inscrits", (F.col("centre_nb_voix") / F.col("nb_inscrits")) * 100) \
    .withColumn("cdroit_pourcentage_voix_inscrits", (F.col("cdroit_nb_voix") / F.col("nb_inscrits")) * 100) \
    .withColumn("droite_pourcentage_voix_inscrits", (F.col("droite_nb_voix") / F.col("nb_inscrits")) * 100) \
    .withColumn("edroite_pourcentage_voix_inscrits", (F.col("edroite_nb_voix") / F.col("nb_inscrits")) * 100)\
    .withColumn("pourcentage_voix_inscrits", (F.col("nb_voix") / F.col("nb_inscrits")) * 100).withColumn("annee", col("annee").cast("long"))


In [34]:
df_election = df_election.withColumn("annee", col("annee").cast("long"))

In [35]:
# popcommunes.csv
df_demographie = pd.read_csv('https://www.dropbox.com/scl/fi/qayze0v5xxwolw4tj8qmc/popcommunes.csv.zip?rlkey=qnpfvfq0kc096dykwfvz2jrv0&st=gk60a3ff&dl=1', compression='zip', header=0, sep=',', quotechar='"')

# cspcommunes.csv
df_csp = pd.read_csv('https://www.dropbox.com/scl/fi/opg91qcti1ctwmopr4efj/cspcommunes.csv.zip?rlkey=h8pti3tzoo7tqjctc2h1rfeqx&st=1mnddf1h&dl=1', compression='zip', header=0, sep=',', quotechar='"')

# df_criminalite = pd.read_csv('https://drive.google.com/uc?export=download&id=1F9Jm0UaemicMy4MnvZ47nmZHiIdsmRwC')

/tmp/ipykernel_96790/8778096.py:2: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demographie = pd.read_csv('https://www.dropbox.com/scl/fi/qayze0v5xxwolw4tj8qmc/popcommunes.csv.zip?rlkey=qnpfvfq0kc096dykwfvz2jrv0&st=gk60a3ff&dl=1', compression='zip', header=0, sep=',', quotechar='"')
/tmp/ipykernel_96790/8778096.py:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csp = pd.read_csv('https://www.dropbox.com/scl/fi/opg91qcti1ctwmopr4efj/cspcommunes.csv.zip?rlkey=h8pti3tzoo7tqjctc2h1rfeqx&st=1mnddf1h&dl=1', compression='zip', header=0, sep=',', quotechar='"')


In [36]:
# Using pandas

df_demographie_filtre = df_demographie.filter(regex="^(dep|nomdep|codecommune|nomcommune|reg|nomreg)$|^pop[0-9]{4}$")

df_csp_filtre = df_csp.filter(regex="^(dep|nomdep|codecommune|nomcommune|agri|indp|cadr|pint|empl|ouvr|pact|chom)(\d{4})?$")

In [37]:
del df_csp
del df_demographie

In [38]:
# Using pySpark

df_demographie_filtre = spark.createDataFrame(df_demographie_filtre)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Could not convert '28' with type str: tried to convert to int64
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [39]:
df_csp_filtre = spark.createDataFrame(df_csp_filtre)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Could not convert '29' with type str: tried to convert to int64
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [40]:
def getColumnsWithStringAndFourDigits(str, df):
    return [col for col in df.columns if col.startswith(str) and len(col) == len(str)+4 and col[-4:].isdigit()]

In [41]:
# cspcommunes

from collections import OrderedDict

# 1. Optimize commune dimension table creation by dropping duplicates
df_dimension_commune = df_demographie_filtre.select("dep","nomdep", "codecommune", 'nomcommune', "reg", "nomreg").join(
                        df_csp_filtre.select("dep","nomdep", "codecommune", 'nomcommune'), on=["dep", "codecommune", "nomdep", 'nomcommune'], how="outer").drop_duplicates().dropna(subset=["reg", "nomreg"]).cache()

# 2. Extract population columns (columns starting with "pop" followed by 4 digits)
pop_columns = getColumnsWithStringAndFourDigits("pop", df_demographie_filtre)

csp_columns = OrderedDict()
for str in ["agri","indp","cadr","pint","empl","ouvr","pact", "chom"]:
    csp_columns[str] = getColumnsWithStringAndFourDigits(str, df_csp_filtre)

annees_csp = OrderedDict()
for key, col_list in csp_columns.items():
    for col_name in col_list:
        annees_csp.setdefault(col_name[-4:], []).append(col_name) 


In [42]:

# 3. Use stack() to pivot the population columns more efficiently for large datasets
df_faits_population = df_demographie_filtre.select("codecommune", F.expr("stack({0}, {1})".format(
                                                        len(pop_columns), ','.join([f"'{col[-4:]}', {col}" for col in pop_columns])
                                        )).alias("annee", "population"))


cast="CAST("
as_double=" AS DOUBLE)"

df_faits_csp = df_csp_filtre.select("codecommune", 
                                F.expr("stack({0}, {1})".format(len(annees_csp),
                                ','.join([f"'{annee}', {', '.join([cast + s + as_double for s in col_name])}" for annee, col_name in annees_csp.items()])
                                )).alias("annee", "agriculteurs", "independants", "cadres", "intermediaires", "employes", "ouvriers", "population_active_totale", "chomeurs"))

df_faits = df_faits_csp.join(df_faits_population, on=["codecommune", "annee"], how="outer").cache()

In [43]:
df_faits_csp.unpersist()
del df_faits_csp
del df_faits_population
df_csp_filtre.unpersist()
del df_csp_filtre
df_demographie_filtre.unpersist()
del df_demographie_filtre
del annees_csp
del csp_columns
del pop_columns
del cast
del as_double

In [44]:
df_dimension_commune = df_dimension_commune.withColumn("codecommune", col("codecommune").cast("long")).withColumn("dep", col("dep").cast("long"))

In [45]:
df_faits = df_faits.na.replace(float("nan"), None)
df_faits = df_faits.withColumn("codecommune", col("codecommune").cast("long")).withColumn("annee", col("annee").cast("long"))

In [46]:
df_faits = df_faits.join(df_election, on=["codecommune", "annee"], how="outer")

In [47]:
df_election.unpersist()
del df_election

In [48]:
df_faits = df_faits.join(delinquance_brut, on=["codecommune", "annee"], how="outer")

In [49]:
delinquance_brut.unpersist()
del delinquance_brut

In [50]:
df_faits = df_faits.join(df_dimension_commune, on=["codecommune"], how="outer")

In [51]:
df_dimension_commune.unpersist()
del df_dimension_commune

In [54]:
import pyspark.sql.types as ty
df_faits = df_faits.withColumn("agriculteurs", col("agriculteurs").cast("integer")) \
                    .withColumn("independants", col("independants").cast("integer"))\
                    .withColumn("cadres", col("cadres").cast("integer"))\
                    .withColumn("intermediaires", col("intermediaires").cast("integer"))\
                    .withColumn("employes", col("employes").cast("integer"))\
                    .withColumn("ouvriers", col("ouvriers").cast("integer"))\
                    .withColumn("population_active_totale", col("population_active_totale").cast("integer"))\
                    .withColumn("chomeurs", col("chomeurs").cast("long"))\
                    .withColumn("nb_inscrits", col("nb_inscrits").cast("integer"))\
                    .withColumn("nb_voix", col("nb_voix").cast("integer"))\
                    .withColumn("autre_nb_voix", col("autre_nb_voix").cast("integer"))\
                    .withColumn("egauche_nb_voix", col("egauche_nb_voix").cast("integer"))\
                    .withColumn("gauche_nb_voix", col("gauche_nb_voix").cast("integer"))\
                    .withColumn("cgauche_nb_voix", col("cgauche_nb_voix").cast("integer"))\
                    .withColumn("centre_nb_voix", col("centre_nb_voix").cast("integer"))\
                    .withColumn("cdroit_nb_voix", col("cdroit_nb_voix").cast("integer"))\
                    .withColumn("droite_nb_voix", col("droite_nb_voix").cast("integer"))\
                    .withColumn("edroite_nb_voix", col("edroite_nb_voix").cast("integer"))\
                    .withColumn("pourcentage_voix_inscrits", col("pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("egauche_pourcentage_voix_inscrits", col("egauche_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("gauche_pourcentage_voix_inscrits", col("gauche_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("cgauche_pourcentage_voix_inscrits", col("cgauche_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("centre_pourcentage_voix_inscrits", col("centre_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("cdroit_pourcentage_voix_inscrits", col("cdroit_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("droite_pourcentage_voix_inscrits", col("droite_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("edroite_pourcentage_voix_inscrits", col("edroite_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("autre_pourcentage_voix_inscrits", col("autre_pourcentage_voix_inscrits").cast("float"))\
                    .withColumn("annee", col("annee").cast(ty.ShortType()))\
                    .withColumn("codecommune", col("codecommune").cast("integer"))\
                    .withColumn("dep", col("dep").cast(ty.ShortType()))\
                    .withColumn("reg", col("reg").cast(ty.ShortType()))\
                    .withColumn("nomcommune", col("nomcommune").cast(ty.VarcharType(50)))\
                    .withColumn("nomdep", col("nomdep").cast(ty.VarcharType(50)))\
                    .withColumn("nomreg", col("nomreg").cast(ty.VarcharType(50)))\
                    .withColumn("type_election", col("type_election").cast(ty.VarcharType(10)))\
                    .withColumn("tour", col("tour").cast(ty.VarcharType(5)))\
                    .withColumn("crimes_delits", col("crimes_delits").cast("integer"))\
                    .withColumn("population", col("population").cast("integer"))

In [55]:
from pyspark.sql.window import Window

df_faits = df_faits.withColumn("id_commune", F.dense_rank().over(Window.orderBy("codecommune"))) \
                    .withColumn("id_type_election", F.dense_rank().over(Window.orderBy("type_election","tour")))\
                    .withColumn("id_demographie", F.dense_rank().over(Window.orderBy("population")))\
                    .withColumn("id_criminalite", F.dense_rank().over(Window.orderBy("crimes_delits")))\
                    .withColumn("id_emploi", F.dense_rank().over(Window.orderBy("agriculteurs", "independants", "cadres", "intermediaires", "employes", "ouvriers", "population_active_totale", "chomeurs")))\
                    .withColumn("id_annee", F.dense_rank().over(Window.orderBy("annee")))\
                    .withColumn("id_annee", col("id_annee").cast(ty.ShortType()))\
                    .withColumn("id_election", col("id_election").cast(ty.VarcharType(15)))\
                    .withColumn("id_type_election", col("id_type_election").cast(ty.ShortType()))

In [56]:
election_columns = ["id_election","id_commune","id_type_election","id_demographie","id_criminalite","id_emploi","id_annee","nb_inscrits","nb_voix","pourcentage_voix_inscrits","autre_nb_voix","egauche_nb_voix",
                    "gauche_nb_voix","cgauche_nb_voix","centre_nb_voix","cdroit_nb_voix","droite_nb_voix","edroite_nb_voix","autre_pourcentage_voix_inscrits","egauche_pourcentage_voix_inscrits","gauche_pourcentage_voix_inscrits",
                    "cgauche_pourcentage_voix_inscrits","centre_pourcentage_voix_inscrits","cdroit_pourcentage_voix_inscrits","droite_pourcentage_voix_inscrits","edroite_pourcentage_voix_inscrits"]
commune_columns = ["id_commune","codecommune","dep","reg","nomcommune","nomdep","nomreg"]
type_election_columns = ["id_type_election","type_election","tour"]
demographie_columns = ["id_demographie","population"]
criminalite_columns = ["id_criminalite","crimes_delits"]
emploi_columns = ["id_emploi","agriculteurs","independants","cadres", "intermediaires", "employes", "ouvriers", "population_active_totale", "chomeurs"]
annee_columns = ["id_annee","annee"]

In [57]:
df_election = df_faits.select(election_columns).distinct()
df_commune = df_faits.select(commune_columns).distinct()
df_type_election = df_faits.select(type_election_columns).distinct()
df_demographie = df_faits.select(demographie_columns).distinct()
df_criminalite = df_faits.select(criminalite_columns).distinct()
df_emploi = df_faits.select(emploi_columns).distinct()
df_annee = df_faits.select(annee_columns).distinct()

In [58]:
df_faits.unpersist()
del df_faits
del election_columns
del commune_columns
del type_election_columns
del demographie_columns
del criminalite_columns
del emploi_columns
del annee_columns

# Écriture des données transformées en base de données avec le driver JDBC

In [59]:
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [60]:
df_election.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="resultats_elections_nuance", mode="overwrite", properties=properties)

In [61]:
df_election.unpersist()
del df_election

In [62]:
df_commune.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="commune", mode="overwrite", properties=properties)

In [63]:
df_commune.unpersist()
del df_commune

In [64]:
df_type_election.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="type_election", mode="overwrite", properties=properties)

In [65]:
df_type_election.unpersist()
del df_type_election

In [66]:
df_demographie.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="demographie", mode="overwrite", properties=properties)

In [67]:
df_demographie.unpersist()
del df_demographie

In [68]:
df_criminalite.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="criminalite", mode="overwrite", properties=properties)

In [69]:
df_criminalite.unpersist()
del df_criminalite

In [70]:
df_emploi.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="emploi", mode="overwrite", properties=properties)

In [71]:
df_emploi.unpersist()
del df_emploi

In [72]:
df_annee.write.jdbc(url="jdbc:mysql://mysql:3306/mspr1", table="annee", mode="overwrite", properties=properties)

In [73]:
df_annee.unpersist()
del df_annee

In [74]:
spark.stop()

# Générer une table au format CSV avec une description statistique du DataFrame

# Effectuer un échantillonage du DataFrame pour en observer les colonnes qui peuvent sembler pertinentes

# Extraire les valeurs uniques d'une colonne afin d'en confirmer la pertinence (colonne ingredients_analysis_tags en exemple)

# Sélection des colonnes nécessaires après étude de celles-ci

# Qualité des données

## On supprime les lignes qui ont moins de 15 colonnes très pertinentes non nulles

## On supprime les lignes dupliquées et on garde le DF en cache pour éviter de le recalculer

## Plus besoin du DataFrame de base donc on le retire de la mémoire cache